In [ ]:
# default_exp losses

# losses

> API details.

In [ ]:
#export
from torch import nn
import torch

In [ ]:
#export
class VILoss(nn.Module):
    """
    Calculate the Kullback-Leibler divergence loss.
    Parameters
    ----------
    model : dies.embedding
        the given embedding to base the loss on
    lambd : float
        scalar for the loss
    """

    def __init__(
        self,
        model,
        base_loss=torch.nn.MSELoss(),
        kl_weight=0.1,
        scale_log_likelihood=True
    ):
        super(VILoss, self).__init__()
        self.base_loss = base_loss
        self.model = model
        self.kl_weight = kl_weight
        self.scale_log_likelihood=scale_log_likelihood

    def forward(self, y, y_hat):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor with the same shape as 'y'

        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """
        base_loss = self.base_loss(y, y_hat)

        n_samples = max(len(y), 1)

        if self.scale_log_likelihood:
            base_loss = base_loss * n_samples

        kl = self.model.nn_kl_divergence()

        loss = base_loss + self.kl_weight * kl

        return loss

    def __repr__(self):
        s = "VILoss(\n  (base_loss):" + str(self.base_loss) + f"\n  (kl_weight): {self.kl_weight} \n)"
        return s

In [ ]:
#export
class Quantile_Score(torch.nn.Module):
    
    def reshape_1(self, x):
        return x.view(x.size()[0],1)
    
    def __init__(self, taus=[0.25, 0.5, 0.75]):
        super(Quantile_Score, self).__init__()
        
        self.taus =  torch.autograd.Variable(torch.tensor(taus, dtype=torch.float32), 
                                             requires_grad=False)
        
        self.taus = self.reshape_1(self.taus).t()
        
    def forward(self, y_hat, y):
        """
        Example:
            y = np.array([0.2, 0.1, 0.3, 0.4])
            tau=np.array([0.25,0.5,0.75])
            for each sample we have one row 
            y_hat = np.array([[0, 0.2, 0.3], 
                          [0.05, 0.1, 0.35], 
                          [0.2, 0.3, 0.6],
                          [0.3, 0.4, 0.45],])
            res = array([0.125 , 0.2   , 0.25  , 0.0625])
        """
        
        y = self.reshape_1(y)
        v = y - y_hat
        
        r = (torch.abs(v*(v>0).float()) * self.taus + \
             torch.abs(v*(v<0).float()) * (1-self.taus))

        # this would calculate the loss for each sample
        # r =  torch.sum(r,dim=1)
        r =  torch.sum(r)
        
        return r


In [ ]:
loss = Quantile_Score(taus=[0.25, 0.5, 0.75]).to("cpu")
probabilistic_forecasts = torch.tensor([[1,2,3],[4,5,6],[7,8,9],])
measurements = torch.tensor([2,5,8])
loss(probabilistic_forecasts, measurements)

tensor(1.5000)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()